In [48]:
import sys  
sys.path.insert(0, './src')
from src import config 
import torch.nn as nn
import os
import torch
from src.model import BERTBaseUncased
device = torch.device('cuda')

In [49]:
model = BERTBaseUncased()
# model = nn.DataParallel(model)
model.load_state_dict(torch.load(os.path.join('src',config.MODEL_PATH)))
model.to(device);

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [50]:
def sentence_prediction(sentence):
    model.eval()
    tokenizer = config.TOKENIZER
    max_length = config.MAX_LENGTH
    review = str(sentence)
    review = " ".join(review.split())
    inputs = tokenizer.encode_plus(
            review,
            None, 
            add_special_tokens = True,
            max_length = max_length,
            pad_to_max_length=True
            )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']

    ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0) #unsqueeze to make batch_size 1
    mask =  torch.tensor(mask, dtype=torch.long).unsqueeze(0)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0)

    
    ids = ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
#     targets = targets.to(device, dtype=torch.float)
    outputs = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
                )
    outputs = torch.sigmoid(outputs)
    return outputs[0][0].cpu().detach().numpy()

In [51]:
input_sentence = "This isn't just a beautifully crafted gangster film. Or an outstanding family portrait, \
                 for that matter. An amazing period piece. A character study. A lesson in filmmaking and an \
                 inspiration to generations of actors, directors, screenwriters and producers. For me, this is more: \
                 this is the definitive film. 10 stars out of 10."
sentence_prediction(input_sentence)

array(0.48731622, dtype=float32)